<a href="https://colab.research.google.com/github/zoraizmohammad/dukeClasses/blob/main/sta240/finalProject/STA240FinalSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***STA240 Final Project***
Completed by: Mohammad Zoraiz & Sylvester Johannes Arizie


# ***General Tasks***
This project is intended as a simulation case study that applies the concepts you learned in class as well as the computational tools you learned in lab. Much of your work will be in coding up a simulation framework that realistically simulates the arrival, service and waiting
times of customers in your business. You might wish to try the following steps to set up the simulator for the base scenario 1 (see below):
1. Code a subprocedure that simulates the arrival times of customers throughout the day.
2. Code a subprocedure that, given output from (a), simulates the serving times of these
customers before they leave.
3. Given output from (a) and (b), code a subprocedure that tells you at a given time point, how many customers are waiting in line, how many customers are being served, and whether the restaurant is experiencing down-time.
4. You can build upon these subprocedures to construct more elaborate and realistic simulation workflows for your restaurant, e.g., Scenario 2 and other scenarios you wish to explore.

Students in prior classes have found it useful to drop by office hours to seek help on coding, so you are more than welcome to do that – we’re here to help!

Project Preview: https://drive.google.com/file/d/1IqlXxooZe8gVvy2u_euSSERWRLfRDYrz/view?usp=sharing

# **Scenario  1**
Consider the simple setting with one dining table and one chef, with operating
hours 10am - 10pm. Suppose customers arrive according to a Poisson process with a rate of λ<sub>A</sub> = 5 per hour. Once a customer arrives, their total service time (ordering, cooking and eating) can be modeled by an exponential distribution with rate λ<sub>S</sub> = 6.

*Note:* It is reasonable to assume waiting and service times for each customer are independently distributed. If a customer arrives and the dining table is occupied, then they would wait in line until the table is vacant.



# **Scenario 2**

Consider the setting with 5 dining tables and L chefs, with operating hours 10am - 10pm. Suppose customers arrive according to a Poisson process with a rate of λ<sub>A</sub> = 10 per hour. Once a customer arrives, their total service time can be modeled by an exponential distribution with rate λ<sub>S</sub> = 3L. Assume a customer spends 50 dollars per meal, and each chef earns a wage of 40 dollars per hour.

Plausible questions: Comment on the customer’s queueing experience in this setting, particularly we vary the number of chefs L. From a business perspective, how many chefs L should be hired to maximize profits with high probability? Justify your recommendation. Comment on the down-time of the restaurant throughout the day, and what this might means for business operation.

# **More Possible Senarios to Consider**
Here are the 10 extensions with specific details added to guide the scenarios:

1. **Variable Customer Arrival Rates**:
   - Assume peak hours are 12:00 PM - 2:00 PM and 6:00 PM - 8:00 PM with a customer arrival rate of 15 customers/hour (Poisson process). Off-peak hours have a rate of 5 customers/hour. Analyze how these fluctuations impact waiting times and required staff levels.

2. **Customer Groups and Table Sizes**:
   - Assume 40% of arrivals are groups of 2, 30% groups of 4, and 30% single customers. Each group requires appropriately sized tables (2-seater, 4-seater). Simulate how accommodating these groups affects wait times and table utilization.

3. **Menu Complexity and Service Time**:
   - Introduce two menu types: a "Basic Menu" with an average service time of 15 minutes (Exponential, rate = 4) and a "Gourmet Menu" with an average service time of 30 minutes (Exponential, rate = 2). Determine how changing 50% of the orders to the Gourmet Menu affects overall service efficiency.

4. **Special Events or Promotions**:
   - Simulate a Friday night "Happy Hour" from 5:00 PM - 7:00 PM with an increased arrival rate of 20 customers/hour. Customers during this period spend an average of 30 dollars per meal instead of 50 dollars. Evaluate its profitability compared to regular evenings.

5. **Customer Patience and Abandonment**:
   - Assume customers are willing to wait up to 10 minutes on average (Exponential patience distribution with rate = 0.1). After this, 25% of them leave without being served. Analyze the impact of adding an extra chef to reduce abandonment rates.

6. **Multi-Level Staff Management**:
   - Introduce a model with 2 waitstaff for every chef, where waitstaff handle seating and order-taking, reducing the chef's service time from 15 minutes to 10 minutes on average (Exponential, rate = 6). Analyze how this division of labor impacts profits and downtime.

7. **Dynamic Pricing**:
   - Assume dynamic pricing increases meal costs by 20% during peak hours (lunch and dinner) and reduces them by 10% during off-peak hours. Evaluate its impact on daily revenue and peak-hour customer retention.

8. **Delivery and Takeout Orders**:
   - Assume 30% of customers during off-peak hours order takeout, requiring 75% of the service time of dine-in customers (Exponential service rate = 8). Analyze the kitchen workload and the effect on dine-in service times.

9. **Seasonal Variations**:
   - Assume summer months (June-August) increase arrival rates to 12 customers/hour on average, while winter months (December-February) reduce them to 8 customers/hour. Evaluate staffing and profitability adjustments required for these changes.

10. **Health and Safety Constraints**:
    - Simulate a social distancing policy limiting occupancy to 3 active dining tables at any time, with one chef per table. Customers have a 20% higher patience threshold (average of 12 minutes). Assess how this restriction affects customer satisfaction and revenue.

## Distribution Information
Here are the extensions with explicit processes and distributions for modeling:

1. **Variable Customer Arrival Rates**:
   - **Arrival Process**: Poisson process with rate λ = 15 customers/hour for peak hours (12 PM - 2 PM and 6 PM - 8 PM), λ = 5 customers/hour for off-peak hours.
   - **Service Times**: Exponential distribution with rate μ = 6 customers/hour.

2. **Customer Groups and Table Sizes**:
   - **Arrival Process**: Poisson process with λ = 10 customers/hour.
   - **Group Sizes**: Discrete distribution: P(single) = 0.3, P(group of 2) = 0.4, P(group of 4) = 0.3.
   - **Service Times**: Exponential distribution with rate μ= 4 (single), 2.5 (group of 2), 1.5 (group of 4).

3. **Menu Complexity and Service Time**:
   - **Arrival Process**: Poisson process with λ = 8 customers/hour.
   - **Service Times**: Mixture of two Exponential distributions: μ1= 4/hour (Basic Menu, probability 0.5), μ2 = 2/hour (Gourmet Menu, probability 0.5).

4. **Special Events or Promotions**:
   - **Arrival Process**: Poisson process with λ = 20 customers/hour during happy hour.
   - **Service Times**: Exponential distribution with rate μ = 5 customers/hour.
   - **Spending per Customer**: Uniform distribution U(25, 35) to model variations in spending during promotions.

5. **Customer Patience and Abandonment**:
   - **Arrival Process**: Poisson process with λ = 10 customers/hour.
   - **Service Times**: Exponential distribution with rate μ = 4 customers/hour.
   - **Patience Times**: Exponential distribution with rate v = 0.1 (mean patience = 10 minutes).

6. **Multi-Level Staff Management**:
   - **Arrival Process**: Poisson process with λ = 12 customers/hour.
   - **Service Times**: Exponential distribution with rate μ = 6 customers/hour when chefs are supported by waitstaff, μ = 4/hour without waitstaff.

7. **Dynamic Pricing**:
   - **Arrival Process**: Poisson process with λ = 15 customers/hour during peak hours,λ= 8 during off-peak.
   - **Service Times**: Exponential distribution with rate μ = 6/hour.
   - **Revenue per Customer**: Piecewise function: $60/customer (peak hours), $45/customer (off-peak).

8. **Delivery and Takeout Orders**:
   - **Arrival Process**: Poisson process with λ= 10 customers/hour.
   - **Takeout Proportion**: Bernoulli distribution with p = 0.3.
   - **Service Times**: Exponential distribution with rate μ = 6/hour (dine-in), μ = 8/hour (takeout).

9. **Seasonal Variations**:
   - **Arrival Process**: Poisson process with λ= 12 customers/hour (summer), λ = 8/hour (winter).
   - **Service Times**: Exponential distribution with rateμ = 6/hour.

10. **Health and Safety Constraints**:
    - **Arrival Process**: Poisson process with λ = 10 customers/hour.
    - **Service Times**: Exponential distribution with rate λ = 6/hour.
    - **Occupancy Constraint**: Uniform distribution U(2, 3)to randomly limit table availability at any given time.

These choices ensure alignment with probabilistic modeling, where Poisson processes are widely used for arrival times and exponential distributions for service times due to their memoryless properties, which are realistic for queueing systems.